In [1]:
type Nm = String

data Ty = TV Nm     -- type variable
        | INT       -- integer type
        | Ty :-> Ty -- function type
      deriving (Eq,Ord,Show)

일치화 규칙

$\displaystyle
[\texttt{INT}=\texttt{INT},~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[t_1\,\text{:->}\,s_1=t_2\,\text{:->}\,s_2,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_2,s_1=s_2,u_2=u_2',~\cdots~],\sigma \\
[x=x,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[x=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{x\mapsto t\}\,[u_2=u_2',~\cdots~],\{x\mapsto t\}\uplus\sigma \\
[t=x,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[x=t,u_2=u_2',~\cdots~],\sigma
$

$\{x\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = [\{x\mapsto t\}\,u_1=\{x\mapsto t\}\,s_1,\cdots,\{x\mapsto t\}\,u_1=\{x\mapsto t\}\,s_1]$

In [9]:
subst (x,t) ty@(TV y)
            | x == y    = t
            | otherwise = ty
subst (x,t) INT         = INT
subst (x,t) (t1 :-> t2) = subst (x,t) t1 :-> subst (x,t) t2

type Eqn = (Ty,Ty)
type Subst = [(Nm,Ty)]

unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((INT    , INT    ): es, s) = unif (es, s)
unif ((t1:->s1, t2:->s2): es, s) = unif ((t1,t2):(s1,s2):es, s)
unif ((TV x   , TV x'  ): es, s) | x==x' = unif (es, s)
unif ((TV x   , t      ): es, s) = unif (es', (x,t):s)
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
unif ((t      , TV x   ): es, s) = unif ((TV x,t):es, s)
unif (es, s) = (es, s)